In [1]:
!pip install evaluate sacrebleu bert_score rouge_score datasets accelerate transformers sentence_transformers openai > /dev/null

In [ ]:
!pip install vec2text

In [4]:
import os
import math
import torch
import openai
import vec2text
import numpy as np
import pandas as pd
from openai import OpenAI
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.chdir('/content/drive/Shareddrives/Strawberries')

client = OpenAI(
  api_key=''
)

os.environ["OPENAI_API_KEY"] =''

corrector = vec2text.load_pretrained_corrector("text-embedding-ada-002")

def get_embeddings_openai(text_list, model="text-embedding-ada-002"):
    batches = math.ceil(len(text_list) / 128)
    outputs = []
    for batch in range(batches):
        # print which batch we're in
        print(f"Processing batch {batch + 1} of {batches}")
        text_list_batch = text_list[batch * 128 : (batch + 1) * 128]
        response = client.embeddings.create(
            input=text_list_batch,
            model=model,
            encoding_format="float",  # override default base64 encoding...
        )
        outputs.extend([e.embedding for e in response.data])
    return torch.tensor(outputs)

Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.20k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

In [ ]:
# Create ada embeddings
path = "Text experiment/textPhenotypes.csv"
sentences = list(pd.read_csv(path, header=0)['AI_Description'].to_numpy())

embeddings = get_embeddings_openai(sentences)
embeddings_df = pd.DataFrame(embeddings.numpy())
embeddings_df.to_csv(f'Text experiment/embeddings.csv', index=False, header=False)

In [8]:
n = 100

# Load text phenotypes
path = "Text experiment/textPhenotypes.csv"
sentences = list(pd.read_csv(path, header=0)['AI_Description'].to_numpy())

embeddings_df = pd.read_csv(f'Text experiment/embeddings.csv', header=None)

embeddings = torch.tensor(embeddings_df.to_numpy()).type(torch.float32)[0:n]

batch_size = 100
inverted_sentences = []

for i in range(0, len(embeddings), batch_size):
  print(f"Processing batch {i // batch_size + 1} of {math.ceil(len(embeddings)/batch_size)}")

  torch.cuda.empty_cache()
  inverted_sentences_batch = vec2text.invert_embeddings(
      embeddings=embeddings[i:i+batch_size].cuda(),
      corrector=corrector,
      num_steps=5,
      sequence_beam_width=2
  )

  inverted_sentences.extend(inverted_sentences_batch)

df = pd.DataFrame({'Original Sentence': sentences[:n], 'Encoded-decoded Sentence': inverted_sentences})
df

Processing batch 1 of 1


,Original Sentence,Encoded-decoded Sentence
0,"Long, deep red strawberry.","Long, deep red strawberry."
1,"Short, pale, bluish-green.","Short, pale, bluish-green."
2,"Medium-long, deep red strawberry.","Medium-long, deep red strawberry."
3,"Long, light red strawberry","Long, light red strawberry"
4,"Long, light red strawberry.","Long, light red strawberry."
...,...,...
95,"Long, deep red strawberry.","Long, deep red strawberry."
96,"Long, deep red strawberry.","Long, deep red strawberry."
97,"Long, light red strawberry.","Long, light red strawberry."
98,"Long, deep red strawberry","Long, deep red strawberry"


Decoding predicted embeddings

In [ ]:
path = "Text experiment/textPhenotypes.csv"
sentences = list(pd.read_csv(path, header=0)['AI_Description'].to_numpy())

n = 563
batch_size = 100
for random_seed in range(14, 51):
  print(random_seed)
  predicted_df = pd.read_csv(f"Text experiment/{random_seed}_rrBLUPpredictedEmbeddings.csv")
  predicted_embeddings = torch.tensor(predicted_df.to_numpy()).type(torch.float32)[0:n]
  inverted_sentences = []
  for i in range(0, len(predicted_embeddings), batch_size):
    print(f"Processing batch {i // batch_size + 1} of {math.ceil(len(predicted_embeddings)/batch_size)}")
    torch.cuda.empty_cache()
    inverted_sentences_batch = vec2text.invert_embeddings(
        embeddings=predicted_embeddings[i:i+batch_size].cuda(),
        corrector=corrector,
        num_steps=5,
        sequence_beam_width=2
    )
    inverted_sentences.extend(inverted_sentences_batch)
  df = pd.DataFrame({'original': sentences[0:n], 'predicted': list(inverted_sentences)[0:n]})
  # save as csv
  df.to_csv(f"Text experiment/{random_seed}_rrBLUPpredictedText.csv", index=False)

14
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
15
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
16
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
17
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
18
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
19
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
20
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 

FileNotFoundError: [Errno 2] No such file or directory: 'Text experiment/46_rrBLUPpredictedEmbeddings.csv'

In [ ]:
df